# Entrena varios modelos de clasificación para predecir is_hit
## utilizando las mismas transformaciones que en 03_preprocessing.

In [1]:
import os

import joblib
import numpy as np
import pandas as pd

from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

## 1. CARGA DEL DATASET LIMPIO (salida lógica de 01_loader)

In [2]:
DATA_PATH = "../data/processed/spotify_clean.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró el archivo en {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset cargado: {df.shape[0]:,} filas x {df.shape[1]} columnas")
display(df.head())

✅ Dataset cargado: 232,724 filas x 15 columnas


,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence,is_hit
0,Movie,0,0.611,0.389,99373,0.910,0.000,0.3460,-1.828,1,0.0525,166.969,4,0.814,0
1,Movie,1,0.246,0.590,137373,0.737,0.000,0.1510,-5.559,0,0.0868,174.003,4,0.816,0
2,Movie,3,0.952,0.663,170267,0.131,0.000,0.1030,-13.879,0,0.0362,99.488,5,0.368,0
3,Movie,0,0.703,0.240,152427,0.326,0.000,0.0985,-12.178,1,0.0395,171.758,4,0.227,0
4,Movie,4,0.950,0.331,82625,0.225,0.123,0.2020,-21.150,1,0.0456,140.576,4,0.390,0


# -------------------------------------------------------
# 2. TRANSFORMACIONES COHERENTES CON 03_preprocessing
##    - Normalización de duración a minutos
##   - Eliminación de duration_ms
# -------------------------------------------------------


In [3]:
print("\nAplicando normalización de duración (como en 03_preprocessing)...")
df["duration_min"] = df["duration_ms"] / 60000
df.drop(columns="duration_ms", inplace=True)

print("Columnas actuales:")
print(df.columns.tolist())



Aplicando normalización de duración (como en 03_preprocessing)...
Columnas actuales:
['genre', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'is_hit', 'duration_min']


# -------------------------------------------------------
# 3. DEFINICIÓN DE X E Y (igual que en 03_preprocessing)
##    - Objetivo: is_hit
##    - Se excluye popularity del set de entrada
# -------------------------------------------------------


In [4]:
X = df.drop(columns=["is_hit", "popularity"])
y = df["is_hit"]

print("\nDimensiones de X e y:")
print("X:", X.shape)
print("y:", y.shape)


Dimensiones de X e y:
X: (232724, 13)
y: (232724,)


# -------------------------------------------------------
# 4. ENCODING DE 'genre' CON LabelEncoder (como en 03_preprocessing)
#    - Se crea X_tree con genre codificado en números
# -------------------------------------------------------

In [5]:

X_tree = X.copy()
le_genre = LabelEncoder()
X_tree["genre"] = le_genre.fit_transform(X_tree["genre"])

print("\nEjemplos de géneros codificados:")
print(X_tree["genre"].unique()[:10])


Ejemplos de géneros codificados:
[15 18  0  1  8  9 10  2 11  3]


# -------------------------------------------------------
# 5. DIVISIÓN TRAIN/TEST (mismo random_state=42)
# -------------------------------------------------------


In [6]:
X_train_tree, X_test_tree, y_train, y_test = train_test_split(
    X_tree,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # estratificado para respetar proporción de hits
)

print("\nTamaños de los conjuntos:")
print("X_train_tree:", X_train_tree.shape)
print("X_test_tree :", X_test_tree.shape)
print("y_train     :", y_train.shape)
print("y_test      :", y_test.shape)


Tamaños de los conjuntos:
X_train_tree: (186179, 13)
X_test_tree : (46545, 13)
y_train     : (186179,)
y_test      : (46545,)


# -------------------------------------------------------
# 6. CÁLCULO DEL POS_WEIGHT PARA MANEJAR DESBALANCEO
#    (como ya hacías en 03_preprocessing)
# -------------------------------------------------------


In [7]:
num_pos = (y_train == 1).sum()
num_neg = (y_train == 0).sum()
pos_weight = num_neg / num_pos

print(f"\n⚖️ Desbalanceo:")
print(f"Positivos (hits)   : {num_pos}")
print(f"Negativos (no hit) : {num_neg}")
print(f"pos_weight         : {pos_weight:.2f}")


⚖️ Desbalanceo:
Positivos (hits)   : 8435
Negativos (no hit) : 177744
pos_weight         : 21.07


# -------------------------------------------------------
# 7. DEFINICIÓN DE MODELOS TIPO ÁRBOL (coherentes con 03_preprocessing)
# -------------------------------------------------------


In [8]:
modelos = {
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=2,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced"
    ),
    "GradientBoosting": GradientBoostingClassifier(
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        n_jobs=-1,
        eval_metric="logloss",
        random_state=42,
        scale_pos_weight=pos_weight
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.9,
        colsample_bytree=0.9,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced"
    )
}

# -------------------------------------------------------
# 8. ENTRENAMIENTO Y EVALUACIÓN EN TEST (F1 COMO MÉTRICA PRINCIPAL)
#    - No se fija ningún resultado “a mano”: todo se calcula desde los datos.
# -------------------------------------------------------


In [9]:
resultados = []

for nombre, modelo in modelos.items():
    print(f"\n🚀 Entrenando modelo: {nombre}")
    modelo.fit(X_train_tree, y_train)

    y_pred = modelo.predict(X_test_tree)
    # Para ROC AUC necesitamos probabilidades de la clase positiva
    if hasattr(modelo, "predict_proba"):
        y_proba = modelo.predict_proba(X_test_tree)[:, 1]
    else:
        # algunos modelos pueden no tener predict_proba; aquí lo dejamos simple
        y_proba = None

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    if y_proba is not None:
        roc = roc_auc_score(y_test, y_proba)
    else:
        roc = np.nan

    resultados.append({
        "Modelo": nombre,
        "Accuracy": acc,
        "F1": f1,
        "ROC_AUC": roc
    })

    print(f"   Accuracy: {acc:.3f}")
    print(f"   F1-score: {f1:.3f}")
    if not np.isnan(roc):
        print(f"   ROC AUC : {roc:.3f}")

df_resultados = pd.DataFrame(resultados).sort_values(by="F1", ascending=False)
print("\n=== Resultados ordenados por F1 (métrica principal) ===")
display(df_resultados)



🚀 Entrenando modelo: RandomForest
   Accuracy: 0.962
   F1-score: 0.320
   ROC AUC : 0.951

🚀 Entrenando modelo: GradientBoosting
   Accuracy: 0.955
   F1-score: 0.017
   ROC AUC : 0.882

🚀 Entrenando modelo: XGBoost
   Accuracy: 0.831
   F1-score: 0.316
   ROC AUC : 0.924

🚀 Entrenando modelo: LightGBM
[LightGBM] [Info] Number of positive: 8435, number of negative: 177744
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020335 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2582
[LightGBM] [Info] Number of data points in the train set: 186179, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
   Accuracy: 0.885
   F1-score: 0.407
   ROC AUC : 0.950

=== Resultados ordenados por F1 (métrica principal) ===


,Modelo,Accuracy,F1,ROC_AUC
3,LightGBM,0.885015,0.407309,0.950258
0,RandomForest,0.961951,0.320154,0.950863
2,XGBoost,0.830680,0.316182,0.924152
1,GradientBoosting,0.954925,0.016870,0.882331


# -------------------------------------------------------
# 9. SELECCIÓN DEL MEJOR MODELO SEGÚN F1
# -------------------------------------------------------


In [10]:
mejor_fila = df_resultados.iloc[0]
mejor_modelo_nombre = mejor_fila["Modelo"]
print(f"\n🏆 Mejor modelo (por F1): {mejor_modelo_nombre}")

mejor_modelo = modelos[mejor_modelo_nombre]


🏆 Mejor modelo (por F1): LightGBM


# -------------------------------------------------------
# 10. ENTRENAR MEJOR MODELO CON TODOS LOS DATOS (X_tree, y)
#     Y GUARDAR MODELO + ENCODER PARA LA API Y EL DASHBOARD
# -------------------------------------------------------


In [11]:
print("\nReentrenando mejor modelo con TODOS los datos disponibles...")
mejor_modelo.fit(X_tree, y)

os.makedirs("../models", exist_ok=True)

MODEL_PATH = "../models/spotify_hit_classifier.joblib"
ENCODER_PATH = "../models/genre_label_encoder.joblib"

joblib.dump(mejor_modelo, MODEL_PATH)
joblib.dump(le_genre, ENCODER_PATH)

print(f"\n💾 Modelo guardado en: {MODEL_PATH}")
print(f"💾 LabelEncoder de género guardado en: {ENCODER_PATH}")


Reentrenando mejor modelo con TODOS los datos disponibles...
[LightGBM] [Info] Number of positive: 10544, number of negative: 222180
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008572 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2582
[LightGBM] [Info] Number of data points in the train set: 232724, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000

💾 Modelo guardado en: ../models/spotify_hit_classifier.joblib
💾 LabelEncoder de género guardado en: ../models/genre_label_encoder.joblib
